In [1]:
# Importando as bibliotecas
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.9.3
urllib -> 3.8
pandas -> 1.2.4


In [6]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(400)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/casas/?pagina=' + str(i) + '&transacao=Venda&tipoUnidade=Residencial,Casa&tipo=Im%C3%B3vel%20usado'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())


            
        cards.append(card)


        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-casas-geral.csv', sep=';' , index=False, encoding='utf-8-sig')
dataset

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade
0,1350000,Água de mina (dentro da propriedade) Mata nat...,Centro,São Bento do Sapucaí,2,4,3,200,0,0,Super destaque
1,229900,A casa no bairro Conjunto Residencial Galo Bra...,Rua Hamilton de Freitas,Conjunto Residencial Galo Branco,2,2,1,60,55,0,Super destaque
2,1000000,"Casa Sitio 05 quartos, 02 suíte 300 m² de área...",Estrada Cachoeira Tapuio,Centro,5,5,3,300,0,0,Super destaque
3,350000,"Casa à Venda, 100m² por R$ 350,000. Turístico ...",Parque Turistico,Peruíbe,3,3,2,100,0,0,Nenhuma
4,270000,"Casa à Venda, 125m² por R$ 270,000. Parque Ban...",Rua João Bento de Almeida,Parque Bandeirantes I,3,4,1,125,0,0,Nenhuma
...,...,...,...,...,...,...,...,...,...,...,...
9595,1200000,Linda casa assobradada de 140 m² de áre constr...,Rua Jupuruchita,Móoca,3,2,3,140,250,0,Nenhuma
9596,848000,"Excelente Casa de Condomínio, no Conjunto Resi...",Mandaqui,São Paulo,4,2,3,220,198,150,Nenhuma
9597,1000000,Sobrado 350 m² de área construída em terreno 2...,Vila Nova Cachoeirinha,São Paulo,4,4,4,350,345,0,Nenhuma
9598,590000,OPORTUNIDADE! Casa à venda na Vila Guilherme e...,Vila Guilherme,São Paulo,2,2,2,165,68,0,Nenhuma


In [5]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(350)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/apartamentos/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

            
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
        cards.append(card)

        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-ap-geral.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade
0,1200000,Alugo e vendo um lindo apartamento no bairro P...,Praia do Pecado,Macaé,3,2,3,135,1300,900,Super destaque
1,1060000,"Apartamento em excelente estado, com terraço, ...",Rua Doutor Veiga Filho,Higienópolis,3,1,2,132,395,2100,Super destaque
2,790000,Cód: NIAP10444. Ao lado da Praça e Metro da No...,Rua Prudente de Morais,Ipanema,1,0,2,45,1060,900,Super destaque
3,235000,O apartamento no bairro Canhema possui 43 metr...,Avenida Fábio Eduardo Ramos Esquivel,Canhema,2,1,1,43,70,195,Destaque
4,320000,O apartamento no bairro Luzia possui 78 metros...,Luzia,Aracaju,3,2,3,78,0,416,Nenhuma
...,...,...,...,...,...,...,...,...,...,...,...
8395,650000,Apartamento Parque Residencial Aquarius Re...,Rua Benedito Osvaldo Lecques,Parque Residencial Aquarius,3,2,2,78,67,420,Nenhuma
8396,310000,"Belíssimo Apto no Jd. Celeste, 2 Dormitórios c...",Rua André Pujos,Jardim Celeste,2,1,1,49,0,450,Destaque
8397,889000,A Vila de Imbassaí está localizado a apenas 70...,Imbassai,Mata de São João,3,4,2,117,1,1,Destaque
8398,440000,"Apartamento à venda na região do Parque Prado,...",Avenida Maria Emília Alves dos Santos de Ângelis,Parque Prado,3,2,2,77,0,849,Nenhuma


In [4]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(400)


##interando por todas as paginas do site
for i in range(pages):
    
    ##Obtendo o HTML
    response = 'https://www.zapimoveis.com.br/venda/casas/rj+rio-de-janeiro/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='simple-card__box')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quarto-min'], card['quarto-max'] = '1','1' 
        elif '-' in quartos.get_text() :
            card['quarto-min'], card['quarto-max'], *outro = quartos.get_text().replace(' ','').replace('\n','').split('-')
        else:        
           card['quarto-min'], card['quarto-max'] = quartos.get_text().replace(' ','').replace('\n',''),quartos.get_text().replace(' ','').replace('\n','')    
        
        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem-min'], card['garagem-max'] = '0' , '0'
        elif '-' in garagem.get_text() :
            card['garagem-min'], card['garagem-max'] = garagem.get_text().replace(' ','').replace('\n','').split('-')
        else:    
            card['garagem-min'], card['garagem-max'] = garagem.get_text().replace(' ','').replace('\n','') , garagem.get_text().replace(' ','').replace('\n','')

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro-min'], card['banheiro-max'] = '1','1'
        elif '-' in banheiro.get_text() :
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n','').split('-')
        else:    
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n',''),banheiro.get_text().replace(' ','').replace('\n','')

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)-min'], card['area(m2)-max'] = '10','30'
        elif '-' in metro.get_text() :
            card['area(m2)-min'], card['area(m2)-max'], *outro = metro.get_text().replace(' ','').replace('\n','').replace('m²','').split('-')
        else:
            card['area(m2)-min'], card['area(m2)-max'] = metro.get_text().replace(' ','').replace('\n','').replace('m²',''),metro.get_text().replace(' ','').replace('\n','').replace('m²','')

            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
        cards.append(card)

        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-casas-rj.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

,valor,descricao,local,bairro/cidade,quarto-min,quarto-max,garagem-min,garagem-max,banheiro-min,banheiro-max,area(m2)-min,area(m2)-max,iptu,condominio,novidade
0,6850000,Casa única no Alto Jardim Botânico! Uma casa ...,Rua Peri,Jardim Botânico,4,4,2,2,8,8,508,610,0,0,Nenhuma
1,1999000,O Condomínio Residencial Camboata está a pouco...,Rua Geógrafo Amora,Piratininga,3,5,3,4,4,6,400,700,0,0,Nenhuma
2,2990000,Mansões Rio Mar. Excelentes projetos de casas ...,Avenida das Américas,Barra da Tijuca,5,5,3,3,6,6,460,540,0,0,Nenhuma
3,510000,"Casa linear, toda independente com quintal na ...",Rua Urucuia,Vila Valqueire,2,2,2,2,2,2,100,100,90,0,Nenhuma
4,1050000,Espetacular casa triplex luxuosa com fino acab...,Estrada do Rio Grande,Taquara,4,4,4,4,4,4,467,467,0,300,Nenhuma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,1930000,,Catete,Rio de Janeiro,11,11,3,3,6,6,380,380,5620,0,Nenhuma
9596,1420000,UNITÁ | VENDA | CASA | 4 QUARTOS | ALVORADA PR...,Avenida Aldemir Martins,Recreio Dos Bandeirantes,4,4,1,1,4,4,194,194,4820,800,Nenhuma
9597,3370000,"(Código: 25135, Rua Campo Belo) Localização ...",Rua Campo Belo,Laranjeiras,4,4,3,3,4,4,356,356,655,0,Nenhuma
9598,3800000,"Maravilhosa casa na Urca, um dos bairros mais ...",Urca,Rio de Janeiro,5,5,3,3,5,5,300,300,1,0,Nenhuma
